## SRGAN Implementation
For deep learning course

#### Dataset Folder


In [ ]:
train_set = TrainDatasetFromFolder('data/DIV2K_train_HR', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set = ValDatasetFromFolder('data/DIV2K_valid_HR', upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)

#### Network Architecture
Network architecutre is defined in **model.py**

It includes:
- Generator(Residual Network)
- Discriminator(VGG Network)

#### Dataset Pre processing
Pre-processing is defined in **data_utils.py**

It includes:
- Image transform 
- Dataset processing

#### Loss Function
Loss functions for training the network

In [26]:
import torch
from torch import nn
from torchvision.models.vgg import vgg16


class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg16(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss
        perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss


class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]


if __name__ == "__main__":
    g_loss = GeneratorLoss()
    print(g_loss)

#### Network Training
Specify training option

In [32]:
# Network Training option
# =======================
# CROPS_SIZE = Training images crop size
# UPSCALE_FACTOR = Super resolution scaling
# NUM_EPOCHS = Number of training epoch

CROP_SIZE = int(64)
UPSCALE_FACTOR = int(4)
NUM_EPOCHS = int(50)

netG = Generator(UPSCALE_FACTOR)
print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
netD = Discriminator()
print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))

generator_criterion = GeneratorLoss()

# Netowrk optimizer
optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

if torch.cuda.is_available():
    netG.cuda()
    netD.cuda()
    generator_criterion.cuda()

# generator parameters: 734219
# discriminator parameters: 5215425


In [ ]:
import argparse
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

import pytorch_ssim
from data_utils import TrainDatasetFromFolder, ValDatasetFromFolder, display_transform
#from loss import GeneratorLoss
from model import Generator, Discriminator


results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

for epoch in range(1, NUM_EPOCHS + 1):
    train_bar = tqdm(train_loader)
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

    netG.train()
    netD.train()
    for data, target in train_bar:
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target)
        if torch.cuda.is_available():
            real_img = real_img.cuda()
        z = Variable(data)
        if torch.cuda.is_available():
            z = z.cuda()
        fake_img = netG(z)

        netD.zero_grad()
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()


        optimizerG.step()

        # loss for current batch before optimization
        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f' % (
            epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']))

   

    # save model parameters
    torch.save(netG.state_dict(), 'epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    torch.save(netD.state_dict(), 'epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    










  0%|                                                                                          | 0/251 [00:00<?, ?it/s]







[1/50] Loss_D: 1.0046 Loss_G: 0.0928 D(x): 0.5012 D(G(z)): 0.3755:   0%|                       | 0/251 [00:03<?, ?it/s]







[1/50] Loss_D: 1.0046 Loss_G: 0.0928 D(x): 0.5012 D(G(z)): 0.3755:   0%|               | 1/251 [00:03<16:25,  3.94s/it]







[1/50] Loss_D: 0.9904 Loss_G: 0.1070 D(x): 0.4498 D(G(z)): 0.3246:   0%|               | 1/251 [00:04<16:25,  3.94s/it]







[1/50] Loss_D: 0.9904 Loss_G: 0.1070 D(x): 0.4498 D(G(z)): 0.3246:   1%|               | 2/251 [00:04<12:06,  2.92s/it]







[1/50] Loss_D: 0.9643 Loss_G: 0.0927 D(x): 0.4331 D(G(z)): 0.2973:   1%|               | 2/251 [00:05<12:06,  2.92s/it]







[1/50] Loss_D: 0.9643 Loss_G: 0.0927 D(x): 0.4331 D(G(z)): 0.2973:   1%|▏              | 3/251 [00:05<09:07,  2.21s/it]







[1/50] Loss_D: 0.9599 Loss_G: 0.0811 D(x): 0.4224 D(G(z)): 0.2738:   1%|▏              | 3/251 [00:05<0

[1/50] Loss_D: 0.8226 Loss_G: 0.0352 D(x): 0.3635 D(G(z)): 0.1303:  11%|█▌            | 27/251 [00:17<02:01,  1.85it/s]







[1/50] Loss_D: 0.8143 Loss_G: 0.0347 D(x): 0.3660 D(G(z)): 0.1320:  11%|█▌            | 27/251 [00:18<02:01,  1.85it/s]







[1/50] Loss_D: 0.8143 Loss_G: 0.0347 D(x): 0.3660 D(G(z)): 0.1320:  11%|█▌            | 28/251 [00:18<02:00,  1.86it/s]







[1/50] Loss_D: 0.8096 Loss_G: 0.0342 D(x): 0.3685 D(G(z)): 0.1305:  11%|█▌            | 28/251 [00:19<02:00,  1.86it/s]







[1/50] Loss_D: 0.8096 Loss_G: 0.0342 D(x): 0.3685 D(G(z)): 0.1305:  12%|█▌            | 29/251 [00:19<01:59,  1.86it/s]







[1/50] Loss_D: 0.8116 Loss_G: 0.0337 D(x): 0.3730 D(G(z)): 0.1263:  12%|█▌            | 29/251 [00:19<01:59,  1.86it/s]







[1/50] Loss_D: 0.8116 Loss_G: 0.0337 D(x): 0.3730 D(G(z)): 0.1263:  12%|█▋            | 30/251 [00:19<01:57,  1.88it/s]







[1/50] Loss_D: 0.8092 Loss_G: 0.0332 D(x): 0.3704 D(G(z)): 0.1253:  12%|█▋            | 30/251 [00:20<01:57,  1

[1/50] Loss_D: 0.7312 Loss_G: 0.0271 D(x): 0.4064 D(G(z)): 0.1029:  22%|███           | 54/251 [00:32<01:45,  1.86it/s]







[1/50] Loss_D: 0.7290 Loss_G: 0.0270 D(x): 0.4106 D(G(z)): 0.1061:  22%|███           | 54/251 [00:32<01:45,  1.86it/s]







[1/50] Loss_D: 0.7290 Loss_G: 0.0270 D(x): 0.4106 D(G(z)): 0.1061:  22%|███           | 55/251 [00:32<01:44,  1.87it/s]







[1/50] Loss_D: 0.7280 Loss_G: 0.0268 D(x): 0.4147 D(G(z)): 0.1092:  22%|███           | 55/251 [00:33<01:44,  1.87it/s]







[1/50] Loss_D: 0.7280 Loss_G: 0.0268 D(x): 0.4147 D(G(z)): 0.1092:  22%|███           | 56/251 [00:33<01:44,  1.87it/s]







[1/50] Loss_D: 0.7286 Loss_G: 0.0267 D(x): 0.4161 D(G(z)): 0.1117:  22%|███           | 56/251 [00:34<01:44,  1.87it/s]







[1/50] Loss_D: 0.7286 Loss_G: 0.0267 D(x): 0.4161 D(G(z)): 0.1117:  23%|███▏          | 57/251 [00:34<01:43,  1.87it/s]







[1/50] Loss_D: 0.7274 Loss_G: 0.0265 D(x): 0.4187 D(G(z)): 0.1133:  23%|███▏          | 57/251 [00:34<01:43,  1

[1/50] Loss_D: 0.7378 Loss_G: 0.0238 D(x): 0.4630 D(G(z)): 0.1707:  32%|████▌         | 81/251 [00:46<01:29,  1.90it/s]







[1/50] Loss_D: 0.7395 Loss_G: 0.0237 D(x): 0.4633 D(G(z)): 0.1767:  32%|████▌         | 81/251 [00:47<01:29,  1.90it/s]







[1/50] Loss_D: 0.7395 Loss_G: 0.0237 D(x): 0.4633 D(G(z)): 0.1767:  33%|████▌         | 82/251 [00:47<01:29,  1.89it/s]







[1/50] Loss_D: 0.7424 Loss_G: 0.0236 D(x): 0.4666 D(G(z)): 0.1837:  33%|████▌         | 82/251 [00:47<01:29,  1.89it/s]







[1/50] Loss_D: 0.7424 Loss_G: 0.0236 D(x): 0.4666 D(G(z)): 0.1837:  33%|████▋         | 83/251 [00:47<01:28,  1.89it/s]







[1/50] Loss_D: 0.7451 Loss_G: 0.0235 D(x): 0.4701 D(G(z)): 0.1901:  33%|████▋         | 83/251 [00:48<01:28,  1.89it/s]







[1/50] Loss_D: 0.7451 Loss_G: 0.0235 D(x): 0.4701 D(G(z)): 0.1901:  33%|████▋         | 84/251 [00:48<01:27,  1.90it/s]







[1/50] Loss_D: 0.7478 Loss_G: 0.0234 D(x): 0.4733 D(G(z)): 0.1961:  33%|████▋         | 84/251 [00:48<01:27,  1

[1/50] Loss_D: 0.8024 Loss_G: 0.0219 D(x): 0.5171 D(G(z)): 0.2980:  43%|█████▌       | 108/251 [01:01<01:17,  1.85it/s]







[1/50] Loss_D: 0.8045 Loss_G: 0.0218 D(x): 0.5179 D(G(z)): 0.3010:  43%|█████▌       | 108/251 [01:01<01:17,  1.85it/s]







[1/50] Loss_D: 0.8045 Loss_G: 0.0218 D(x): 0.5179 D(G(z)): 0.3010:  43%|█████▋       | 109/251 [01:01<01:16,  1.86it/s]







[1/50] Loss_D: 0.8062 Loss_G: 0.0218 D(x): 0.5197 D(G(z)): 0.3047:  43%|█████▋       | 109/251 [01:02<01:16,  1.86it/s]







[1/50] Loss_D: 0.8062 Loss_G: 0.0218 D(x): 0.5197 D(G(z)): 0.3047:  44%|█████▋       | 110/251 [01:02<01:16,  1.84it/s]







[1/50] Loss_D: 0.8076 Loss_G: 0.0217 D(x): 0.5205 D(G(z)): 0.3071:  44%|█████▋       | 110/251 [01:02<01:16,  1.84it/s]







[1/50] Loss_D: 0.8076 Loss_G: 0.0217 D(x): 0.5205 D(G(z)): 0.3071:  44%|█████▋       | 111/251 [01:02<01:15,  1.86it/s]







[1/50] Loss_D: 0.8092 Loss_G: 0.0216 D(x): 0.5218 D(G(z)): 0.3103:  44%|█████▋       | 111/251 [01:03<01:15,  1

...

[4/50] Loss_D: 1.0010 Loss_G: 0.0098 D(x): 0.4702 D(G(z)): 0.4705:  22%|███           | 55/251 [00:36<01:51,  1.76it/s]







[4/50] Loss_D: 1.0010 Loss_G: 0.0098 D(x): 0.4702 D(G(z)): 0.4705:  22%|███           | 56/251 [00:36<01:49,  1.77it/s]







[4/50] Loss_D: 1.0010 Loss_G: 0.0098 D(x): 0.4716 D(G(z)): 0.4719:  22%|███           | 56/251 [00:36<01:49,  1.77it/s]







[4/50] Loss_D: 1.0010 Loss_G: 0.0098 D(x): 0.4716 D(G(z)): 0.4719:  23%|███▏          | 57/251 [00:36<01:49,  1.78it/s]







[4/50] Loss_D: 1.0008 Loss_G: 0.0098 D(x): 0.4723 D(G(z)): 0.4725:  23%|███▏          | 57/251 [00:37<01:49,  1.78it/s]







[4/50] Loss_D: 1.0008 Loss_G: 0.0098 D(x): 0.4723 D(G(z)): 0.4725:  23%|███▏          | 58/251 [00:37<01:48,  1.78it/s]







[4/50] Loss_D: 1.0006 Loss_G: 0.0098 D(x): 0.4733 D(G(z)): 0.4732:  23%|███▏          | 58/251 [00:37<01:48,  1.78it/s]







[4/50] Loss_D: 1.0006 Loss_G: 0.0098 D(x): 0.4733 D(G(z)): 0.4732:  24%|███▎          | 59/251 [00:37<01:48,  1

[4/50] Loss_D: 1.0016 Loss_G: 0.0095 D(x): 0.4849 D(G(z)): 0.4855:  33%|████▌         | 82/251 [00:51<01:34,  1.79it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0095 D(x): 0.4849 D(G(z)): 0.4855:  33%|████▋         | 83/251 [00:51<01:34,  1.77it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0095 D(x): 0.4857 D(G(z)): 0.4864:  33%|████▋         | 83/251 [00:51<01:34,  1.77it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0095 D(x): 0.4857 D(G(z)): 0.4864:  33%|████▋         | 84/251 [00:51<01:33,  1.79it/s]







[4/50] Loss_D: 1.0015 Loss_G: 0.0095 D(x): 0.4862 D(G(z)): 0.4867:  33%|████▋         | 84/251 [00:52<01:33,  1.79it/s]







[4/50] Loss_D: 1.0015 Loss_G: 0.0095 D(x): 0.4862 D(G(z)): 0.4867:  34%|████▋         | 85/251 [00:52<01:33,  1.78it/s]







[4/50] Loss_D: 1.0015 Loss_G: 0.0095 D(x): 0.4869 D(G(z)): 0.4876:  34%|████▋         | 85/251 [00:53<01:33,  1.78it/s]







[4/50] Loss_D: 1.0015 Loss_G: 0.0095 D(x): 0.4869 D(G(z)): 0.4876:  34%|████▊         | 86/251 [00:53<01:32,  1

[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4960 D(G(z)): 0.4961:  43%|█████▋       | 109/251 [01:06<01:22,  1.72it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4960 D(G(z)): 0.4961:  44%|█████▋       | 110/251 [01:06<01:20,  1.75it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4962 D(G(z)): 0.4963:  44%|█████▋       | 110/251 [01:07<01:20,  1.75it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4962 D(G(z)): 0.4963:  44%|█████▋       | 111/251 [01:07<01:19,  1.76it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4967 D(G(z)): 0.4967:  44%|█████▋       | 111/251 [01:08<01:19,  1.76it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4967 D(G(z)): 0.4967:  45%|█████▊       | 112/251 [01:08<01:18,  1.76it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4965 D(G(z)): 0.4966:  45%|█████▊       | 112/251 [01:08<01:18,  1.76it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0094 D(x): 0.4965 D(G(z)): 0.4966:  45%|█████▊       | 113/251 [01:08<01:18,  1

[4/50] Loss_D: 1.0015 Loss_G: 0.0094 D(x): 0.4977 D(G(z)): 0.4980:  54%|███████      | 136/251 [01:22<01:05,  1.75it/s]







[4/50] Loss_D: 1.0015 Loss_G: 0.0094 D(x): 0.4977 D(G(z)): 0.4980:  55%|███████      | 137/251 [01:22<01:05,  1.75it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4974 D(G(z)): 0.4979:  55%|███████      | 137/251 [01:22<01:05,  1.75it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4974 D(G(z)): 0.4979:  55%|███████▏     | 138/251 [01:22<01:04,  1.74it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4974 D(G(z)): 0.4979:  55%|███████▏     | 138/251 [01:23<01:04,  1.74it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4974 D(G(z)): 0.4979:  55%|███████▏     | 139/251 [01:23<01:04,  1.75it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4970 D(G(z)): 0.4974:  55%|███████▏     | 139/251 [01:23<01:04,  1.75it/s]







[4/50] Loss_D: 1.0016 Loss_G: 0.0093 D(x): 0.4970 D(G(z)): 0.4974:  56%|███████▎     | 140/251 [01:23<01:03,  1

[4/50] Loss_D: 1.0012 Loss_G: 0.0093 D(x): 0.4945 D(G(z)): 0.4948:  65%|████████▍    | 163/251 [01:37<00:50,  1.76it/s]







[4/50] Loss_D: 1.0012 Loss_G: 0.0093 D(x): 0.4945 D(G(z)): 0.4948:  65%|████████▍    | 164/251 [01:37<00:49,  1.77it/s]







[4/50] Loss_D: 1.0013 Loss_G: 0.0093 D(x): 0.4946 D(G(z)): 0.4950:  65%|████████▍    | 164/251 [01:38<00:49,  1.77it/s]







[4/50] Loss_D: 1.0013 Loss_G: 0.0093 D(x): 0.4946 D(G(z)): 0.4950:  66%|████████▌    | 165/251 [01:38<00:48,  1.78it/s]







[4/50] Loss_D: 1.0013 Loss_G: 0.0093 D(x): 0.4944 D(G(z)): 0.4949:  66%|████████▌    | 165/251 [01:38<00:48,  1.78it/s]







[4/50] Loss_D: 1.0013 Loss_G: 0.0093 D(x): 0.4944 D(G(z)): 0.4949:  66%|████████▌    | 166/251 [01:38<00:47,  1.79it/s]







[4/50] Loss_D: 1.0014 Loss_G: 0.0093 D(x): 0.4944 D(G(z)): 0.4950:  66%|████████▌    | 166/251 [01:39<00:47,  1.79it/s]







[4/50] Loss_D: 1.0014 Loss_G: 0.0093 D(x): 0.4944 D(G(z)): 0.4950:  67%|████████▋    | 167/251 [01:39<00:46,  1

#### Save training result

In [ ]:
 
parser = argparse.ArgumentParser(description='Train Super Resolution Models')
parser.add_argument('--crop_size', default=88, type=int, help='training images crop size')
parser.add_argument('--upscale_factor', default=4, type=int, choices=[2, 4, 8],
                    help='super resolution upscale factor')
parser.add_argument('--num_epochs', default=100, type=int, help='train epoch number')

# argparse is a command to parse argument from user. In this program,it is used to specify some parameter
# configuration before starting the network training

# __name__ == '__main__' is used to make sure the program is run as a main program. If it is called by other,
# it will not run the code below.
#if __name__ == '__main__':
#opt = parser.parse_args()

#CROP_SIZE = opt.crop_size
#UPSCALE_FACTOR = opt.upscale_factor
#NUM_EPOCHS = opt.num_epochs




    netG.eval()
    out_path = 'training_results/SRF_' + str(UPSCALE_FACTOR) + '/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    with torch.no_grad():
        val_bar = tqdm(val_loader)
        valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
        val_images = []
        for val_lr, val_hr_restore, val_hr in val_bar:
            batch_size = val_lr.size(0)
            valing_results['batch_sizes'] += batch_size
            lr = val_lr
            hr = val_hr
            if torch.cuda.is_available():
                lr = lr.cuda()
                hr = hr.cuda()
            sr = netG(lr)

            batch_mse = ((sr - hr) ** 2).data.mean()
            valing_results['mse'] += batch_mse * batch_size
            batch_ssim = pytorch_ssim.ssim(sr, hr).item()
            valing_results['ssims'] += batch_ssim * batch_size
            valing_results['psnr'] = 10 * log10(1 / (valing_results['mse'] / valing_results['batch_sizes']))
            valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
            val_bar.set_description(
                desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                    valing_results['psnr'], valing_results['ssim']))

            val_images.extend(
                [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
                 display_transform()(sr.data.cpu().squeeze(0))])
        val_images = torch.stack(val_images)
        val_images = torch.chunk(val_images, val_images.size(0) // 15)
        #val_save_bar = tqdm(val_images, desc='[saving training results]')
        #index = 1
        #for image in val_save_bar:
        #    image = utils.make_grid(image, nrow=3, padding=5)
        #    utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
        #    index += 1
# save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(valing_results['psnr'])
    results['ssim'].append(valing_results['ssim'])

    if epoch % 10 == 0 and epoch != 0:
        out_path = 'statistics/'
        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')

#### Network Testing